# Importing packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import time
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

# Loading the Dataset

In [3]:
train_pref = pd.read_csv("C:/Users/Surface/Documents/BDA_106/Data/Train_PreppedData_SummarByAddressNCategory.csv", parse_dates=['Dates'])

(878049, 31)


# Data Cleaning & Exploring

In [4]:
# Dropping the duplicate values
train_pref.drop_duplicates(inplace=True)
# Replacing the wrong Geo_Coordinated
train_pref.replace({'X':-120.5,'Y':90.0},np.NaN,inplace=True)
imp_mean = SimpleImputer(strategy='mean')

for district in train_pref['PdDistrict'].unique():
    train_pref.loc[train_pref['PdDistrict'] == district,['X','Y']] = imp_mean.fit_transform(train_pref.loc[train_pref['PdDistrict'] == district,['X','Y']])

In [6]:
# Converting Categorical value into Numerical value
le3 = LabelEncoder()
train_pref['PdDistrict'] = le3.fit_transform(train_pref['PdDistrict'])
le4 = LabelEncoder()
y_pref = le4.fit_transform(train_pref.pop('Category'))

In [5]:
train_pref['Day_of_Week'] = train_pref['Dates'].dt.weekday
#Dropping the columns
train_pref.drop(columns=['Descript','Resolution','Address','Dates','DayOfWeek'], inplace=True)
train_pref.shape
# Splitting the Dataset into train and test
trai_X, test_X, trai_y, test_y = train_test_split(train_pref, y_pref)

(875726, 27)

# Modelling

In [7]:
# Building Naive Bayes Classifier
start_time = time.time()
GNB = GaussianNB()
GNB.fit(trai_X, trai_y)
predicted_pref = np.array(GNB.predict_proba(test_X))
end_time = time.time()
secs = (end_time - start_time)
loss_function = log_loss(test_y,predicted_pref)
print("Total seconds: {} and loss {}".format(secs, loss_function))

Total seconds: 5.0479371547698975 and loss 3.5474153091697365


In [10]:
# Building Random Forest Classifier
start_time = time.time()
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(trai_X, trai_y)
predicted_pref_rfc = np.array(rfc.predict_proba(test_X))
end_time = time.time()
secs = (end_time - start_time)
loss_function = log_loss(test_y,predicted_pref_rfc)
print("Total seconds: {} and loss {}".format(secs, loss_function))

Total seconds: 341.0735692977905 and loss 0.15475482534447066


In [21]:
# Building K-Nearest Neighbor
start_time = time.time()
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(trai_X, trai_y)
predicted_pref_knn = np.array(knn.predict_proba(test_X))
end_time = time.time()
secs = (end_time - start_time)
loss_function = log_loss(test_y,predicted_pref_knn)
print("Total seconds: {} and loss {}".format(secs, loss_function))

Total seconds: 44.79580330848694 and loss 3.1684245563990228


# Evaluation

In [9]:
# Evaluation for Naive Bayes Model
y_pref_pred = GNB.predict(test_X)
print("Accuracy:",metrics.accuracy_score(test_y, y_pref_pred))

Accuracy: 0.42093892167430985


In [11]:
#Evaluation for Random Forest Classifier
y_pref_pred = rfc.predict(test_X)
print("Accuracy:",metrics.accuracy_score(test_y, y_pref_pred))

Accuracy: 0.982834852831016


In [20]:
#Evaluation for K-nearest Neighbors
y_pref_pred = knn.predict(test_X)
print("Accuracy:",metrics.accuracy_score(test_y, y_pref_pred))

Accuracy: 0.6480779420093911
